# [RNN을 사용한 텍스트 분류](https://www.tensorflow.org/tutorials/text/text_classification_rnn?hl=ko)

In [1]:
!pip install -q tfds-nightly

In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [3]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])
    plt.show()

In [4]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [5]:
# 인코더가 포함되어 있다?
encoder = info.features['text'].encoder

In [6]:
print('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


In [7]:
# 텍스트 인코더는 문자열을 가역적으로 인코딩하여 필요한 경우 바이트 인코딩
sample_string = 'Hello TensorFlow.'

encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print('The original string: "{}"'.format(original_string))

Encoded string is [4025, 222, 6307, 2327, 4043, 2120, 7975]
The original string: "Hello TensorFlow."


In [8]:
assert original_string == sample_string

In [9]:
for index in encoded_string:
    print('{} ----> {}'.format(index, encoder.decode([index])))

4025 ----> Hell
222 ----> o 
6307 ----> Ten
2327 ----> sor
4043 ----> Fl
2120 ----> ow
7975 ----> .


In [10]:
# 훈련을 위한 데이터 준비하기
BUFFER_SIZE = 10000
BATCH_SIZE = 64

# 다음으로 이러한 인코딩된 문자열을 일괄적으로 생성합니다. 
# padded_batch 메서드를 사용하여 배치에서 가장 긴 문자열의 길이로 시퀀스를 0으로 채웁니다.
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE)

test_dataset = test_dataset.padded_batch(BATCH_SIZE)

In [11]:
# 모델 생성하기
model = tf.keras.Sequential([
    # Embedding : 이 인덱스 조회는 원-핫 인코딩된 벡터를 
    # tf.keras.layers.Dense 레이어를 통해 전달하는 동등한 연산보다 훨씬 효율적입니다.
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [12]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [13]:
!ls -l /usr/local | grep cuda

lrwxrwxrwx 1 root root    9 Oct  1  2020 cuda -> cuda-11.0
drwxr-xr-x 1 root root   21 Jan 22 22:39 cuda-11.0


## Tutorial ERROR 로 인한 중지

In [15]:
# 무언가 문제가 있다.
# history = model.fit(train_dataset, epochs=10,
#                     validation_data=test_dataset, 
#                     validation_steps=30)